In [1]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras.models import Sequential, load_model

from keras_self_attention import SeqSelfAttention
from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import keras
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *
from keras.models import model_from_json
import json



Using TensorFlow backend.


In [3]:
def sanitize(string):
    words = string.split(' ')
    return words

In [2]:
!nvidia-smi


Tue Jul  9 02:04:50 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   46C    P8     8W / 200W |    185MiB /  8118MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
#read csv

train = pd.read_csv("dataset/PAWS_QQP/PAWS_QQP_train.tsv", sep='\t')
dev = pd.read_csv("dataset/PAWS_QQP/PAWS_QQP_dev_and_test.tsv", sep='\t')
test = pd.read_csv("dataset/PAWS_QQP/PAWS_QQP_dev_and_test.tsv", sep='\t')


train['title1_tokenized'] = \
    train.loc[:, 'sentence1'] \
         .apply(sanitize)
train['title2_tokenized'] = \
    train.loc[:, 'sentence2'] \
         .apply(sanitize)


dev['title1_tokenized'] = \
    dev.loc[:, 'sentence1'] \
         .apply(sanitize)
dev['title2_tokenized'] = \
    dev.loc[:, 'sentence2'] \
         .apply(sanitize)


test['title1_tokenized'] = \
    test.loc[:, 'sentence1'] \
         .apply(sanitize)
test['title2_tokenized'] = \
    test.loc[:, 'sentence2'] \
         .apply(sanitize)


print(dev[:2])


   id                                          sentence1  \
0   1  b'What were the major effects of the cambodia ...   
1   2  b"The guy I 'm dating never texts me and I fee...   

                                           sentence2  label  \
0  b'What were the major effects of the Iquique e...      0   
1  b"The guy I 'm dating never wants me and I fee...      0   

                                    title1_tokenized  \
0  [b'What, were, the, major, effects, of, the, c...   
1  [b"The, guy, I, 'm, dating, never, texts, me, ...   

                                    title2_tokenized  
0  [b'What, were, the, major, effects, of, the, I...  
1  [b"The, guy, I, 'm, dating, never, wants, me, ...  


In [133]:
print(test[:2])

   id                                          sentence1  \
0   1  b'What were the major effects of the cambodia ...   
1   2  b"The guy I 'm dating never texts me and I fee...   

                                           sentence2  label  \
0  b'What were the major effects of the Iquique e...      0   
1  b"The guy I 'm dating never wants me and I fee...      0   

                                    title1_tokenized  \
0  [b'What, were, the, major, effects, of, the, c...   
1  [b"The, guy, I, 'm, dating, never, texts, me, ...   

                                    title2_tokenized  
0  [b'What, were, the, major, effects, of, the, I...  
1  [b"The, guy, I, 'm, dating, never, wants, me, ...  


In [5]:
MAX_NUM_WORDS = 10000
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words=MAX_NUM_WORDS)

train_corpus_x1 = train.title1_tokenized
train_corpus_x2 = train.title2_tokenized
train_corpus = pd.concat([
    train_corpus_x1, train_corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(train_corpus)
tokenizer.texts_to_sequences(train_corpus)
x1_train = tokenizer \
    .texts_to_sequences(train_corpus_x1)
x2_train = tokenizer \
    .texts_to_sequences(train_corpus_x2)


print(tokenizer.document_count)




dev_corpus_x1 = dev.title1_tokenized
dev_corpus_x2 = dev.title2_tokenized
dev_corpus = pd.concat([
    dev_corpus_x1, dev_corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(dev_corpus)
tokenizer.texts_to_sequences(dev_corpus)
x1_dev = tokenizer \
    .texts_to_sequences(dev_corpus_x1)
x2_dev = tokenizer \
    .texts_to_sequences(dev_corpus_x2)


test_corpus_x1 = test.title1_tokenized
test_corpus_x2 = test.title2_tokenized
test_corpus = pd.concat([
    test_corpus_x1, test_corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(test_corpus)
tokenizer.texts_to_sequences(test_corpus)
x1_test = tokenizer \
    .texts_to_sequences(test_corpus_x1)
x2_test = tokenizer \
    .texts_to_sequences(test_corpus_x2)

word_index = tokenizer.word_index


print(tokenizer.document_count)

23726
26402


In [6]:
max_seq_len1 = max([
    len(seq) for seq in x1_train])
print(max_seq_len1)

50


In [7]:


MAX_SEQUENCE_LENGTH = 50  #better to have words covered than uncovered
x1_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)

x1_dev = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_dev, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2_dev = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_dev, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x1_test = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_test, 
                   maxlen=MAX_SEQUENCE_LENGTH)

x2_test = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_test, 
                   maxlen=MAX_SEQUENCE_LENGTH)


y_train = train.label

y_train = keras \
    .utils \
    .to_categorical(y_train, num_classes=2)


y_dev = dev.label

y_dev = keras \
    .utils \
    .to_categorical(y_dev, num_classes=2)



y_test = test.label

y_test = keras \
    .utils \
    .to_categorical(y_test, num_classes=2)

x1_test[1]


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1850,
        215,    9,  101,  689,  297, 4957,   71,    5,    9,  120,   47,
        108,   28,   54, 1034,   84,   71,   52,   50,    9,  166,  259,
        108,  718,   71,  108, 1037,   71,    5,  853,   71,   20,   59,
         12,    9,  120,   79,  103,   19], dtype=int32)

In [9]:
#add cos similarity value to last elem of word embedding
x1_test_cos=[]
x2_test_cos=[]
x1_train_cos=[]
x2_train_cos=[]
x1_dev_cos=[]
x2_dev_cos=[]

from scipy import spatial

def cos_sim(vec1, vec2):
    return (1 - spatial.distance.cosine(vec1, vec2))

for x in range(len(y_train)):
    
    
    cos_train=cos_sim(x1_train[x], x2_train[x])
    
    
    x1_train_cos.append(np.append(x1_train[x], cos_train))
    x2_train_cos.append(np.append(x2_train[x], cos_train))
    
    


for x in range(len(y_test)):
    cos_dev=cos_sim(x1_dev[x], x2_dev[x])
    cos_test=cos_sim(x1_test[x], x2_test[x])
    x1_test_cos.append(np.append(x1_test[x], cos_test))
    x2_test_cos.append(np.append(x2_test[x], cos_test))
    x1_dev_cos.append(np.append(x1_dev[x], cos_dev))
    x2_dev_cos.append(np.append(x2_dev[x], cos_dev))

In [56]:
len(x1_train_cos)

11863

In [130]:
x1_test[:2]

[[11,
  153,
  2,
  265,
  521,
  10,
  2,
  1441,
  758,
  8,
  5,
  30,
  12,
  236,
  521,
  112,
  3,
  2,
  4981,
  758,
  6,
  4982,
  1],
 [1850,
  215,
  9,
  101,
  689,
  297,
  4957,
  71,
  5,
  9,
  120,
  47,
  108,
  28,
  54,
  1034,
  84,
  71,
  52,
  50,
  9,
  166,
  259,
  108,
  718,
  71,
  108,
  1037,
  71,
  5,
  853,
  71,
  20,
  59,
  12,
  9,
  120,
  79,
  103,
  19]]

In [51]:
#BiLSTM?
NUM_CLASSES=2 #boolean; 1 or 0

#rough estimate

MAX_SEQUENCE_LENGTH = 50 #how long one sentence is

#this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension


NUM_EMBEDDING_DIM = 256

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')


embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)

top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)


shared_lstm = Bidirectional(LSTM(NUM_LSTM_UNITS))
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)


merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 30

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
mcp_save = ModelCheckpoint('BiLSTM_PAWS_QQP.h5', save_best_only=True,  verbose=1, monitor='val_loss', mode='min')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 50, 256)      2560000     input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 256)          394240      embedding_4[0][0]                
          

In [81]:
#cosine similarity


NUM_CLASSES=2 #boolean; 1 or 0

#rough estimate

MAX_SEQUENCE_LENGTH = 50 #how long one sentence is

#this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension


NUM_EMBEDDING_DIM = 256

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')


embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)

top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)


shared_lstm = Bidirectional(LSTM(NUM_LSTM_UNITS))
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)


merged = dot(
    [top_output, bm_output], 
    axes=1, normalize = True)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 40

earlyStopping = EarlyStopping(monitor='val_loss', patience=8, verbose=1, mode='min')
mcp_save = ModelCheckpoint('BiLSTM_PAWS_QQP_cos_sim.h5', verbose=1, save_best_only=True, monitor='val_loss', mode='min')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 51)           0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           (None, 51)           0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 51, 256)      2560000     input_12[0][0]                   
                                                                 input_13[0][0]                   
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) (None, 256)          394240      embedding_6[0][0]                
          

In [121]:
#BiLSTM with last cos_sim element concatenated?
NUM_CLASSES=2 #boolean; 1 or 0

#rough estimate

MAX_SEQUENCE_LENGTH = 51 #how long one sentence is

#this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension


NUM_EMBEDDING_DIM = 256

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')



embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)

top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)



shared_lstm = Bidirectional(LSTM(NUM_LSTM_UNITS))

top_output = shared_lstm(top_embedded)

bm_output = shared_lstm(bm_embedded)




merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 30

earlyStopping = EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='max')
mcp_save = ModelCheckpoint('BiLSTM_PAWS_QQP_concat_sim_cos.h5', save_best_only=True,  verbose=1, monitor='val_acc', mode='max')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_28 (InputLayer)           (None, 51)           0                                            
__________________________________________________________________________________________________
input_29 (InputLayer)           (None, 51)           0                                            
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 51, 256)      2560000     input_28[0][0]                   
                                                                 input_29[0][0]                   
__________________________________________________________________________________________________
bidirectional_14 (Bidirectional (None, 256)          394240      embedding_14[0][0]               
          

In [90]:
#attention model

#last elem cos sim + attention

NUM_CLASSES=2 #boolean; 1 or 0

#rough estimate

MAX_SEQUENCE_LENGTH = 51 #how long one sentence is

#this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension


NUM_EMBEDDING_DIM = 256

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')


embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)

top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)

shared_LSTM = Bidirectional(LSTM(NUM_LSTM_UNITS, return_sequences=True)) 

top_output = shared_LSTM(top_embedded)

bm_output = shared_LSTM(bm_embedded)



top_attention_output = SeqSelfAttention(attention_activation='sigmoid')(top_output)

bm_attention_output = SeqSelfAttention(attention_activation='sigmoid')(bm_output)


merged = concatenate(
    [top_attention_output, bm_attention_output], 
    axis=-1)

#new_merged = keras.layers.Reshape((-1, 2))(merged)

flatten_merged = Flatten()(merged)

dense =  Dense(
    units=2, name='Dense')
predictions = dense(flatten_merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=[predictions])


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 30

earlyStopping = EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='max')
mcp_save = ModelCheckpoint('BiLSTM_PAWS_QQP_simcos_attention.h5', save_best_only=True,  verbose=1, monitor='val_acc', mode='max')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_79 (InputLayer)           (None, 51)           0                                            
__________________________________________________________________________________________________
input_80 (InputLayer)           (None, 51)           0                                            
__________________________________________________________________________________________________
embedding_46 (Embedding)        (None, 51, 256)      2560000     input_79[0][0]                   
                                                                 input_80[0][0]                   
__________________________________________________________________________________________________
bidirectional_50 (Bidirectional (None, 51, 256)      394240      embedding_46[0][0]               
          

In [76]:
#y_train.shape
temp = np.reshape(merged, (-1,2))
temp.shape

ValueError: cannot reshape array of size 1 into shape (2)

In [91]:
history = model.fit(x=[x1_train_cos, x2_train_cos], y=y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, callbacks=[earlyStopping, mcp_save], validation_data=([x1_dev_cos, x2_dev_cos], y_dev),
    
    shuffle=True
)

Train on 11863 samples, validate on 669 samples
Epoch 1/30
11863/11863 [==============================] - 15s 1ms/step - loss: 6.5231 - acc: 0.6843 - val_loss: 4.5535 - val_acc: 0.7175

Epoch 00001: val_acc improved from -inf to 0.71749, saving model to BiLSTM_PAWS_QQP_simcos_attention.h5
Epoch 2/30
11863/11863 [==============================] - 8s 685us/step - loss: 5.0883 - acc: 0.6843 - val_loss: 4.5535 - val_acc: 0.7175

Epoch 00002: val_acc did not improve from 0.71749
Epoch 3/30
11863/11863 [==============================] - 8s 686us/step - loss: 5.0883 - acc: 0.6843 - val_loss: 4.5535 - val_acc: 0.7175

Epoch 00003: val_acc did not improve from 0.71749
Epoch 4/30
11863/11863 [==============================] - 8s 687us/step - loss: 5.0883 - acc: 0.6843 - val_loss: 4.5535 - val_acc: 0.7175

Epoch 00004: val_acc did not improve from 0.71749
Epoch 5/30
11863/11863 [==============================] - 8s 688us/step - loss: 5.0883 - acc: 0.6843 - val_loss: 4.5535 - val_acc: 0.7175

Epoc

Saved model to disk


In [94]:
model = load_model('BiLSTM_PAWS_QQP_concat_sim_cos.h5')


In [95]:

predicts = model.predict(
    [x1_train_cos, x2_train_cos])

In [96]:
predicts[:5]
len(predicts)

11863

In [96]:
real1=train.sentence1
real2=train.sentence2
realCat=train.label
no = []
err1 = []
err2 = []
wrongCat = []


for x in range(len(predicts)):
    if (np.argmax(predicts[x])!=(train.label[x])):
       
            no=np.append(no, x)
            err1=np.append(err1, real1[x])
            err2=np.append(err2, real2[x])
            wrongCat = np.append(wrongCat, realCat[x])
    
        
print(1-(len(wrongCat)/len(predicts)))

0.9075276068448116


In [97]:
#0.7873219253
error_analysis = Table()


error_analysis = error_analysis.with_column("id", no, "sent1", err1, "sent2", err2, "label", wrongCat)


#false POSITIVE: swap two words
#false negative: swap two specific nouns
error_analysis.show(30)

id,sent1,sent2,label
13,b'Are Indian IT services companies like Tech Mahindra/ W ...,b'Are Indian IT services companies like Wipro/TCS/Tech . ...,1
14,b'Is nuclear energy renewable energy ?',"b""`` Is nuclear energy `` renewable energy `` ? ''""",1
19,b'Thinking to start reading novels.Suggest which books t ...,b'Thinking to start reading novels.Suggest which books t ...,1
21,b'How can I make an android app using Python 3 ? Is ther ...,b'How can I develop an android app using Python 3 ? Is t ...,1
28,"b""What does the phrase 'Do you feel me ' mean ?""","b""What does the phrase `Do you feel me'mean ?""",1
29,b'What are some unexpected things first-time visitors to ...,"b'What are some unexpected things , first-time visitors ...",1
35,b'How do I prepare for cognizant campus placement online ...,"b""How do I prepare for Cognizant 's online campus placem ...",1
53,b'How does quality of life in Vancouver compare to that ...,b'How does quality of life in Vancouver compare to that ...,1
61,b'I have only a very nice startup idea but I m not good ...,b'I have only a very good startup idea but I m not nice ...,1
78,"b'Which CPU should I buy , an Intel or an AMD ?'","b'Which CPU should I buy , an AMD or an Intel ?'",1


In [100]:
error_analysis.where("label", are.equal_to(1)).show(30)

id,sent1,sent2,label
13,b'Are Indian IT services companies like Tech Mahindra/ W ...,b'Are Indian IT services companies like Wipro/TCS/Tech . ...,1
14,b'Is nuclear energy renewable energy ?',"b""`` Is nuclear energy `` renewable energy `` ? ''""",1
19,b'Thinking to start reading novels.Suggest which books t ...,b'Thinking to start reading novels.Suggest which books t ...,1
21,b'How can I make an android app using Python 3 ? Is ther ...,b'How can I develop an android app using Python 3 ? Is t ...,1
28,"b""What does the phrase 'Do you feel me ' mean ?""","b""What does the phrase `Do you feel me'mean ?""",1
29,b'What are some unexpected things first-time visitors to ...,"b'What are some unexpected things , first-time visitors ...",1
35,b'How do I prepare for cognizant campus placement online ...,"b""How do I prepare for Cognizant 's online campus placem ...",1
53,b'How does quality of life in Vancouver compare to that ...,b'How does quality of life in Vancouver compare to that ...,1
61,b'I have only a very nice startup idea but I m not good ...,b'I have only a very good startup idea but I m not nice ...,1
78,"b'Which CPU should I buy , an Intel or an AMD ?'","b'Which CPU should I buy , an AMD or an Intel ?'",1
